In [3]:
import pandas as pd
import patsy as pt
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import pymc3 as pm
import matplotlib.ticker as tk
import re
import pickle

WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'


In [15]:
!mkdir outputs/bayes_gp_m52

out_dir = 'outputs/bayes_gp_m52/'

## Import data

In [9]:
df = pd.read_csv('outputs/ala1_trials_clean.csv')
df = df.rename(columns={'project_name': 'basis', 'cluster__n_clusters': 'n', 'test_mean': 'y'}).\
loc[:, ['basis', 'y', 'n']]

## Scale predictors



Note: I'm using a more rational naming for 'n' here, in contrast to the notebooks before. 

In [11]:
to_log = ['n']
for col in to_log: 
    df.loc[:, col+'_log'] = np.log(df[col])

to_scale = ['n_log']
scaler = preprocessing.MinMaxScaler()
vars_scaled = pd.DataFrame(scaler.fit_transform(df.loc[:, to_scale]), columns=[x+'_s' for x in to_scale])
df = df.join(vars_scaled)
df.T

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
basis,psi,rmsd,phipsi,phipsi,phi,positions,rmsd,phi,psi,positions,...,positions,phipsi,phipsi,positions,phipsi,positions,phipsi,positions,positions,positions
y,1.79637,1.70957,3.27798,3.28693,1.98877,3.29715,1.70869,1.98756,1.79831,3.16998,...,3.31788,3.30056,3.29722,3.32348,3.29749,3.33242,3.31647,2.94608,3.31237,3.08385
n,77,554,97,95,362,440,942,169,96,33,...,359,172,656,490,617,955,169,23,763,29
n_log,4.34381,6.31716,4.57471,4.55388,5.89164,6.08677,6.84801,5.1299,4.56435,3.49651,...,5.88332,5.14749,6.48616,6.19441,6.42487,6.86171,5.1299,3.13549,6.63726,3.3673
n_s,0.064843,0.548126,0.0851064,0.08308,0.353597,0.432624,0.941236,0.158055,0.0840932,0.0202634,...,0.350557,0.161094,0.651469,0.483283,0.611955,0.954407,0.158055,0.0101317,0.759878,0.0162107
n_log_s,0.40961,0.864007,0.46278,0.457982,0.766024,0.810956,0.986242,0.59062,0.460393,0.214506,...,0.764108,0.594672,0.902921,0.83574,0.888808,0.989398,0.59062,0.131377,0.937714,0.184753


## Create design matrix

In [13]:
y = df.loc[:, 'y']
X = df.loc[:, df.columns.difference(['y'])]
X_c = pt.dmatrix('~ 0 + n_log_s + C(basis)', data=df, return_type='dataframe')
X_c = X_c.rename(columns=lambda x: re.sub('C|\\(|\\)|\\[|\\]','',x))

## Model fitting functions

In [19]:
def gamma(alpha, beta):
    def g(x):
        return pm.Gamma(x, alpha=alpha, beta=beta)
    return g

def hcauchy(beta):
    def g(x):
        return pm.HalfCauchy(x, beta=beta)
    return g


def fit_model_2(y, X, kernel_type='M32', n=1000, n_chains=2):
    """
    function to return a pymc3 model
    y : dependent variable
    X : independent variables
    prop_Xu : number of inducing varibles to use
    
    X, y are dataframes. We'll use the column names. 
    """
    with pm.Model() as model:
        # Covert arrays
        X_a = X.values
        y_a = y.values
        X_cols = list(X.columns)
        
        # Globals
        prop_Xu = 0.01
        l_prior = gamma(1, 0.05)
        eta_prior = hcauchy(2)
        sigma_prior = hcauchy(2)

        # Kernels
        # 3 way interaction
        eta = eta_prior('eta')
        cov = eta**2
        for i in range(X_a.shape[1]):
            var_lab = 'l_'+X_cols[i]
            if kernel_type=='RBF':
                cov = cov*pm.gp.cov.ExpQuad(X_a.shape[1], ls=l_prior(var_lab), active_dims=[i])
            if kernel_type=='Exponential':
                cov = cov*pm.gp.cov.Exponential(X_a.shape[1], ls=l_prior(var_lab), active_dims=[i])
            if kernel_type=='M52':
                cov = cov*pm.gp.cov.Matern52(X_a.shape[1], ls=l_prior(var_lab), active_dims=[i])
            if kernel_type=='M32':
                cov = cov*pm.gp.cov.Matern32(X_a.shape[1], ls=l_prior(var_lab), active_dims=[i])

        # Covariance model
        cov_tot = cov 

        # Model
        gp = pm.gp.MarginalSparse(cov_func=cov_tot, approx="FITC")

        # Noise model
        sigma_n =sigma_prior('sigma_n')

        # Inducing variables
        num_Xu = int(X_a.shape[0]*prop_Xu)
        Xu = pm.gp.util.kmeans_inducing_points(num_Xu, X_a)

        # Marginal likelihood
        y_ = gp.marginal_likelihood('y_', X=X_a, y=y_a,Xu=Xu, noise=sigma_n)
        trace = pm.sample(draws=n, chains=n_chains, cores=1)
        
    return gp, trace, model


# In[39]:




In [20]:
gp, trace, model = fit_model_2(y=y, X=X_c, kernel_type='M52')
pickle.dump(
    {'gp': gp, 'trace': trace, 'model': model},
    open(out_dir+'model.p', 'wb')
)


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/Users/robertarbon/opt/miniconda3/envs/analysis/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/Users/robertarbon/opt/miniconda3/envs/analysis/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result[diagonal_slice] = x
/Users/robertarbon/opt/miniconda3/envs/analysis/lib/python3.6/site-packages/theano/tensor/basic.py:6611: FutureW

In [33]:
df_trace = pd.DataFrame({x : trace.get_values(x) for x in trace.varnames if x[-5:]!='log__'})
bdf_trace.to_csv(out_dir+'posterior.csv', index=False)

# len_labs = [x for x in list(df_trace.columns) if x[0]=='l']

# relevance = pd.DataFrame(1/df_trace.loc[:, len_labs].values, columns=len_labs)
# relevance_m = relevance.melt(var_name='Hyperparameter', value_name='Relevance')
# with sns.plotting_context('paper', font_scale=1.25):
#     sns.set_style('whitegrid')
#     ax = sns.boxplot(data=relevance_m, x='Relevance', y='Hyperparameter', whis=2)
#     ax.set_xscale('log')
#     ax.xaxis.set_major_formatter(tk.StrMethodFormatter('{x:4.2f}'))
#     ax.xaxis.set_minor_locator(tk.LogLocator(base=10.0, subs='auto', numdecs=4))
#     ax.tick_params(which='minor', axis='x', bottom=True, direction='in')

In [36]:
data = X.join(y)
data.to_csv(out_dir+'data.csv', index=False)

In [37]:
pwd

'/Users/robertarbon/OneDrive - University of Bristol/Research/optimize_msms/Ala1'